In [53]:
import os
import sys
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras


#tf version should be 2.5 or higher
tf.__version__

'2.10.1'

In [54]:
# check if GPU is being used
print("GPU Status: ", end = '')
if len(tf.config.list_physical_devices('GPU')) != 0: print("Active")
else: print("Inactive")

GPU Status: Active


In [64]:
train_dir = os.path.join('C:\\Users\\ptpsm\\.keras\\datasets', 'sign_mnist_train','sign_mnist_train.csv')
test_dir = os.path.join('C:\\Users\\ptpsm\\.keras\\datasets', 'sign_mnist_test','sign_mnist_test.csv')

train_file = tf.keras.utils.get_file('sign_mnist_train.csv', train_dir)
test_file = tf.keras.utils.get_file('sign_mnist_test.csv', test_dir)

train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

train_df.head()
# train_df.dtypes

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [56]:
#scale model
train_raw = train_raw / 255.0
test_raw = test_raw / 255.0

In [58]:
#reshape inputs for CNN layers
train_raw = train_raw.reshape(train_raw.shape[0], 28, 28, 1)
test_raw = test_raw.reshape(test_raw.shape[0], 28, 28, 1)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
# CNN MODEL
#  - Notes - 
# Total Filters (Features) = 108
# Increased stride resolution
# Added an additional conv2d layer
# Regularization Techniques Used:
#   1) Batch Normalization (scale = False due to relu)
#   2) Dropout (decreased to 20% due to batch normalization)

model = keras.Sequential([
    keras.layers.Conv2D(kernel_size = 3, filters = 12, use_bias = False, padding = 'same', input_shape = (28, 28, 1)),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 24, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Conv2D(kernel_size = 6, filters = 32, use_bias = False, padding = 'same', strides = 1),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Flatten(),

    keras.layers.Dense(200, use_bias = False),
    keras.layers.BatchNormalization(center = True, scale = False),
    keras.layers.Activation('relu'),

    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation = 'softmax')
])

In [ ]:
# NEW MODEL COMPILE
#  - Notes - 
# Changed optimizer to adam to maintain momentum
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

In [ ]:
#evaluate
test_loss, test_acc = model.evaluate(test_images,  test_labels)
print('\nTest accuracy:', test_acc)

In [ ]:
# Predict on the first 5 test images.
predictions = model.predict(test_images[:5])

In [ ]:
#Raw predictions
predictions

In [ ]:
# Print our model's predictions
print(np.argmax(predictions, axis=1))

In [ ]:
# Check our predictions against the ground truths
print(test_labels[:5]) # [7, 2, 1, 0, 4]

### (NEW) Evaluate the model

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.95, 1])
plt.legend(loc='lower right')

# Included loss to measure overfitting
plt.figure()
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.2])
plt.legend(loc='upper right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)